In [1]:
import sys
from memory_profiler import profile

sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
import jsonpickle
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.metrics import pairwise
# from MKLpy.metrics import margin, radius, ratio, trace, frobenius
# from MKLpy.preprocessing import kernel_normalization
from MKLpy import generators
from MKLpy.algorithms import AverageMKL, EasyMKL
from sklearn.svm import SVC
from MKLpy.model_selection import cross_val_score
from MKLpy.generators import HPK_generator, RBF_generator
import pandas as pd
from collections import defaultdict
from itertools import product
import os
import torch
import time
import pickle
import new_alternate_single_svm as nalsvm
from mkl_data_processing import storage_location,cross_validation_results_location

In [2]:


def load_pickled_in_filename(file):
    # load a simple pickled file and return it. its a bit different to the method used for the dictionary as this
    # is pure Python 3.x
    pickle_in = open(file, 'rb')
    return pickle.load(pickle_in)

In [12]:
cv_dict_list = defaultdict(dict)
for symbol in ['STAN.L','AAL.L']:
    print(symbol)
    cross_validation_data_location = cross_validation_results_location(symbol)
    clean_data_location = storage_location(symbol)
    for alternate_label_idx in range(0,2):
        alternate_label = nalsvm.labels_pickle_files[alternate_label_idx]
        print(alternate_label)
        file_to_load = os.path.join(clean_data_location, os.listdir(clean_data_location)[alternate_label_idx])
        pkl_file = load_pickled_in_filename(file_to_load)
        date_keys = list(pkl_file.keys())
        print('--------------->',date_keys)
        date=date_keys[5]
        print(date)
        start = time.time()
        nalsvm.logmemoryusage("Before garbage collect")
        Xtr = normalization(rescale_01(torch.Tensor(pkl_file[date][0].values)))
        Ytr = torch.Tensor(pkl_file[date][1].values)
        print('first bit done')
        nalsvm.gc.collect()
        KLrbf = generators.RBF_generator(Xtr, gamma=[.001, .01, .1])
        print('done with kernel')
        nalsvm.gc.collect()
        try:
            lam_values = [0, 0.1, 0.2, 1]
            C_values = [0.01, 1, 100]
            print(C_values)
            for lam, C in product(lam_values, C_values):
                print('now here', C, lam)
                svm = SVC(C=C)
                mkl = EasyMKL(lam=lam, learner=svm)
                scores = cross_val_score(KLrbf, Ytr, mkl, n_folds=3, scoring='accuracy')
                print(str(scores))
                print(lam, C, scores)
                print(type(scores))
                cv_dict_list[(symbol, date, alternate_label)][(lam,C)] = scores
                nalsvm.logmemoryusage("Before garbage collect")

        except (ValueError, TypeError, EOFError):
            continue
        pickle_out_filename = os.path.join(cross_validation_data_location, 'TestCrossValidationResults.pkl')
        test_df =pd.DataFrame.from_dict(cv_dict_list)
        test_df.to_pickle(pickle_out_filename)

        nalsvm.gc.collect()
        print('done too')
        end = time.time()
        print(f'it took {end - start} seconds!')




STAN.L
LabelsAlternateFive
---------------> ['20170703', '20170704', '20170705', '20170706', '20170721', '20170724', '20170801', '20170802', '20170803', '20170804', '20170807', '20170810', '20170811', '20170814', '20170815', '20170821', '20170822', '20170823', '20170825', '20170830', '20170901', '20170904', '20170905', '20170906', '20170908', '20170914', '20170915', '20170918', '20170925', '20170927', '20170928', '20180307', '20180312', '20180313', '20180314', '20180321', '20180327', '20180328', '20180412', '20180413', '20180417', '20180418']
20170724
Memory usage at Before garbage collect is 1643.462656mb.
first bit done
done with kernel
[0.01, 1, 100]
now here 0.01 0
[0.9983779399837794, 0.9983779399837794, 0.9983766233766234]
0 0.01 [0.9983779399837794, 0.9983779399837794, 0.9983766233766234]
<class 'list'>
Memory usage at Before garbage collect is 1692.2255360000001mb.
now here 1 0
[0.9983779399837794, 0.9983779399837794, 0.9983766233766234]
0 1 [0.9983779399837794, 0.9983779399837

In [4]:
cv_dict_list.keys()

dict_keys([('STAN.L', '20170724', 'LabelsAlternateFive'), ('STAN.L', '20170724', 'LabelsAlternateFour')])

In [5]:
svm = SVC(C=C)
mkl = EasyMKL(lam=lam, learner=svm)
Ytr

tensor([ 0.,  0.,  0.,  ...,  1., -1.,  1.])

In [6]:
scores = cross_val_score(KLrbf, Ytr, mkl, n_folds=3, scoring='accuracy')


In [7]:
print('about to dump pickle')

pickle_out = open(pickle_out_filename, 'wb')
pickle.dump(cv_dict_list)
pickle_out.close()

about to dump pickle


TypeError: Required argument 'file' (pos 2) not found

In [13]:
pd.read_pickle(pickle_out_filename)

STAN.L  \
                                                     20170724   
                                          LabelsAlternateFive   
0.0 0.01    [0.9983779399837794, 0.9983779399837794, 0.998...   
    1.00    [0.9983779399837794, 0.9983779399837794, 0.998...   
    100.00  [0.9983779399837794, 0.9983779399837794, 0.998...   
0.1 0.01    [0.9983779399837794, 0.9983779399837794, 0.998...   
    1.00    [0.9983779399837794, 0.9983779399837794, 0.998...   
    100.00  [0.9983779399837794, 0.9983779399837794, 0.997...   
0.2 0.01    [0.9983779399837794, 0.9983779399837794, 0.998...   
    1.00    [0.9983779399837794, 0.9983779399837794, 0.998...   
    100.00  [0.9967558799675588, 0.9983779399837794, 0.998...   
1.0 0.01    [0.9983779399837794, 0.9983779399837794, 0.998...   
    1.00    [0.9983779399837794, 0.9983779399837794, 0.998...   
    100.00  [0.9983779399837794, 0.9983779399837794, 0.998...   

                                                               
                                                               
                                          LabelsAlternateFour  
0.0 0.01    [0.9618815896188159, 0.9610705596107056, 0.961...  
    1.00    [0.9618815896188159, 0.9610705596107056, 0.961...  
    100.00  [0.9618815896188159, 0.9610705596107056, 0.961...  
0.1 0.01    [0.9618815896188159, 0.9610705596107056, 0.961...  
    1.00    [0.9618815896188159, 0.9610705596107056, 0.961...  
    100.00  [0.9618815896188159, 0.9610705596107056, 0.961...  
0.2 0.01    [0.9618815896188159, 0.9610705596107056, 0.961...  
    1.00    [0.9618815896188159, 0.9610705596107056, 0.961...  
    100.00  [0.9618815896188159, 0.9610705596107056, 0.961...  
1.0 0.01    [0.9618815896188159, 0.9610705596107056, 0.961...  
    1.00    [0.9618815896188159, 0.9610705596107056, 0.961...  
    100.00  [0.9618815896188159, 0.9610705596107056, 0.961...

In [10]:
pickle_out_filename

'/media/ak/My Passport/Data/FinDataReal/JointLocationsAlternateDataClean/CV_Results/STAN.L/TestCrossValidationResults.pkl'

In [15]:
for symbol in ['STAN.L','AAL.L']:
    print(symbol)
    cross_validation_data_location = cross_validation_results_location(symbol)
    clean_data_location = storage_location(symbol)
    for alternate_label_idx in range(0,2):
        alternate_label = nalsvm.labels_pickle_files[alternate_label_idx]
        print(alternate_label)
        file_to_load = os.path.join(clean_data_location, os.listdir(clean_data_location)[alternate_label_idx])
        pkl_file = load_pickled_in_filename(file_to_load)
        date_keys = list(pkl_file.keys())
        print('--------------->',date_keys)
        for date in date_keys:
            start = time.time()
            nalsvm.logmemoryusage("Before garbage collect")
            Xtr = normalization(rescale_01(torch.Tensor(pkl_file[date][0].values)))
            Ytr = torch.Tensor(pkl_file[date][1].values)
            print('first bit done')
            nalsvm.gc.collect()
            KLrbf = generators.RBF_generator(Xtr, gamma=[.001, .01, .1])
            print('done with kernel')
            nalsvm.gc.collect()
            try:
                lam_values = [0, 0.1, 0.2, 1]
                C_values = [0.01, 1, 10, 100]
                print(C_values)
                for lam, C in product(lam_values, C_values):
                    print('now here', C, lam)
                    svm = SVC(C=C)
                    mkl = EasyMKL(lam=lam, learner=svm)
                    scores = cross_val_score(KLrbf, Ytr, mkl, n_folds=3, scoring='accuracy')
                    print(str(scores))
                    print(lam, C, scores)
                    print(type(scores))
                    cv_dict_list[(symbol, date, alternate_label)][(lam,C)] = scores
                    nalsvm.logmemoryusage("Before garbage collect")

            except (ValueError, TypeError, EOFError):
                continue
            # only way that seems to work for this
            pickle_out_filename = os.path.join(cross_validation_data_location, "_".join((symbol,date,'RBF_CrossValidationResults.pkl')))
            test_df =pd.DataFrame.from_dict(cv_dict_list)
            test_df.to_pickle(pickle_out_filename)

            nalsvm.gc.collect()
            print('done too')
            end = time.time()
            print(f'it took {end - start} seconds!')




STAN.L
LabelsAlternateFive
---------------> ['20170703', '20170704', '20170705', '20170706', '20170721', '20170724', '20170801', '20170802', '20170803', '20170804', '20170807', '20170810', '20170811', '20170814', '20170815', '20170821', '20170822', '20170823', '20170825', '20170830', '20170901', '20170904', '20170905', '20170906', '20170908', '20170914', '20170915', '20170918', '20170925', '20170927', '20170928', '20180307', '20180312', '20180313', '20180314', '20180321', '20180327', '20180328', '20180412', '20180413', '20180417', '20180418']
Memory usage at Before garbage collect is 1789.89056mb.
first bit done
done with kernel
[0.01, 1, 10, 100]
now here 0.01 0


KeyboardInterrupt: 